# Tutorial 16: Monitoring & Metrics

Learn how to:
- Retrieve embedding/search metrics
- Inspect hybrid search lineage stats
- Tail audit logs for operational awareness

The notebook gracefully handles demo mode when a live API is unavailable.

In [ ]:
from tutorial_utils import ping_server, pretty_json
import requests
from pathlib import Path

API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"\ud83d\udce1 Tensorus server available: {SERVER}")


## Embedding & search metrics
Call `/api/v1/vector/metrics` to inspect aggregate latency and throughput statistics.

In [ ]:
if SERVER:
    try:
        metrics_resp = requests.get(f"{API}/api/v1/vector/metrics", timeout=10)
        metrics_resp.raise_for_status()
        vector_metrics = metrics_resp.json()
    except requests.RequestException as exc:
        vector_metrics = {"error": str(exc), "demo_mode": True}
else:
    vector_metrics = {"demo_mode": True, "embedding_metrics": {"avg_latency_ms": 42}, "search_metrics": {"p95_ms": 88}}

print(pretty_json(vector_metrics))


## Hybrid search lineage stats
Use `/api/v1/vector/metrics` response or `/api/v1/lineage/...` endpoints to understand lineage fan-out and depth.

In [ ]:
if SERVER:
    try:
        lineage_stats_resp = requests.get(f"{API}/api/v1/vector/metrics", timeout=10)
        lineage_stats_resp.raise_for_status()
        lineage_stats = lineage_stats_resp.json().get("lineage_metrics", {})
    except requests.RequestException as exc:
        lineage_stats = {"error": str(exc), "demo_mode": True}
else:
    lineage_stats = {"demo_mode": True, "lineage_metrics": {"avg_steps": 3, "max_depth": 6}}

print(pretty_json(lineage_stats))


## Audit log tailing
Tensorus writes audit events to `tensorus_audit.log`. Tail the log to monitor critical actions.

In [ ]:
audit_path = Path('tensorus_audit.log')
if audit_path.exists():
    tail = ''.join(audit_path.open().readlines()[-5:])
    print(tail)
else:
    print('Demo mode: audit log not found.')
